In [1]:
flow_len = 1

import pandas as pd

df_list = []
for split in ['train', 'valid', 'test']:
    
    print('Current split ', split)
    
    df_tmp = pd.read_csv(split+'_split.csv')
    print(df_tmp.shape)

    with open(split+'_uttr_eng.txt', 'r') as f:
        eng_uttrs = f.readlines()
    print(len(eng_uttrs))

    df_tmp['uttr'] = [i.strip() for i in eng_uttrs]
    df_list.append(df_tmp)

df_all = pd.concat(df_list)
print(df_all.shape)

Current split  train
(94187, 18)
94187
Current split  valid
(11137, 18)
11137
Current split  test
(27438, 18)
27438
(132762, 19)


In [2]:
df_all['A'] = df_all['Agreeableness'].apply(lambda x: 1 if x == 'high' else 0)
df_all['C'] = df_all['Conscientiousness'].apply(lambda x: 1 if x == 'high' else 0)
df_all['E'] = df_all['Extraversion'].apply(lambda x: 1 if x == 'high' else 0)
df_all['O'] = df_all['Openness'].apply(lambda x: 1 if x == 'high' else 0)
df_all['N'] = df_all['Neuroticism'].apply(lambda x: 1 if x == 'high' else 0)
df_all.head()

,TV_ID,Dialogue_ID,Utterance_ID,Speaker,Gender,Age,Neuroticism,Extraversion,Openness,Agreeableness,...,Sentiment,Emotion,DA,Utterance,uttr,A,C,E,O,N
0,1,01_000,01_000_000,童文洁,female,middle-aged,high,high,low,low,...,neutral,neutral,greeting,真巧,what a coincidence,0,1,1,0,1
1,1,01_000,01_000_001,童文洁,female,middle-aged,high,high,low,low,...,neutral,neutral,greeting,车没事了,the car is fine,0,1,1,0,1
2,1,01_000,01_000_002,刘静,female,middle-aged,low,high,high,high,...,neutral,neutral,answer,是你呀 没事了没事 谢谢你,"It's you, it's okay, it's okay, thank you",1,1,1,1,0
3,1,01_000,01_000_003,童文洁,female,middle-aged,high,high,low,low,...,neutral,neutral,question,没事没事 你也去春风,"It's okay, you can go to Chunfeng too",0,1,1,0,1
4,1,01_001,01_001_000,刘静,female,middle-aged,low,high,high,high,...,neutral,neutral,answer,对,right,1,1,1,1,0


In [3]:
columns = ['TV_ID', 'Dialogue_ID', 'Utterance_ID', 'Speaker', 'A', 'C', 'E', 'O', 'N', 'Emotion', 'uttr', 'Sentiment', 'DA']
df_all_clean = df_all[columns]

df_all_clean.head()

,TV_ID,Dialogue_ID,Utterance_ID,Speaker,A,C,E,O,N,Emotion,uttr,Sentiment,DA
0,1,01_000,01_000_000,童文洁,0,1,1,0,1,neutral,what a coincidence,neutral,greeting
1,1,01_000,01_000_001,童文洁,0,1,1,0,1,neutral,the car is fine,neutral,greeting
2,1,01_000,01_000_002,刘静,1,1,1,1,0,neutral,"It's you, it's okay, it's okay, thank you",neutral,answer
3,1,01_000,01_000_003,童文洁,0,1,1,0,1,neutral,"It's okay, you can go to Chunfeng too",neutral,question
4,1,01_001,01_001_000,刘静,1,1,1,1,0,neutral,right,neutral,answer


In [4]:
sample_list = []
cnt_dialog = 0
sent_dialog = []

import statistics

for dialogue in df_all_clean.groupby(['Dialogue_ID']):
    # dialogue_id = dialogue[0]
    cnt_dialog += 1
    dialog_length = len(list(dialogue[1]['Emotion']))
    emotions   = list(dialogue[1]['Emotion'])[:int(flow_len*dialog_length)]
    sentiments = list(dialogue[1]['Sentiment'])[:int(flow_len*dialog_length)]
    DAs        = list(dialogue[1]['DA'])[:int(flow_len*dialog_length)]
    sents      = list(dialogue[1]['uttr'])[:int(flow_len*dialog_length)]
    speakers   = list(dialogue[1]['Speaker'])[:int(flow_len*dialog_length)]
    
    A          = list(dialogue[1]['A'])[:int(flow_len*dialog_length)]
    C          = list(dialogue[1]['C'])[:int(flow_len*dialog_length)]
    E          = list(dialogue[1]['E'])[:int(flow_len*dialog_length)]
    O          = list(dialogue[1]['O'])[:int(flow_len*dialog_length)]
    N          = list(dialogue[1]['N'])[:int(flow_len*dialog_length)]
    
    sent_dialog.append(len(sents))
    
    for speaker in list(set(speakers)):
        dialog_state = [int(speaker == i) for i in speakers]
        if sum(dialog_state) < len(dialog_state)/2:
            continue
        else:
            uttrs = []
            label_a, label_c, label_e, label_o, label_n = 0,0,0,0,0
            for sent, state, a,c,e,o,n in zip(sents, dialog_state, A,C,E,O,N):
                if state == 1:
                    uttrs.append(sent)
                    label_a = a
                    label_c = c
                    label_e = e
                    label_o = o
                    label_n = n
            sample_list.append([speaker, dialog_state, emotions, sentiments, DAs,
                                    '#'.join(sents), ' '.join(uttrs),
                                label_a, label_c, label_e, label_o, label_n])



df_data = pd.DataFrame(sample_list, columns = ['speaker', 'dialog_state', 'emotions', 'sentiment', 'DA', 'sent', 'utterance', 'A','C','E','O','N'])
print(df_data.shape)

df_data.head()

/tmp/ipykernel_3189690/544803416.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for dialogue in df_all_clean.groupby(['Dialogue_ID']):


(13215, 12)


,speaker,dialog_state,emotions,sentiment,DA,sent,utterance,A,C,E,O,N
0,童文洁,"[1, 1, 0]","[neutral, neutral, neutral]","[neutral, neutral, neutral]","[greeting, greeting, answer]","what a coincidence#the car is fine#It's you, i...",what a coincidence the car is fine,0,1,1,0,1
1,童文洁,"[0, 1]","[neutral, neutral]","[neutral, neutral]","[answer, question]",right#Your kids go to school here too,Your kids go to school here too,0,1,1,0,1
2,刘静,"[1, 0]","[neutral, neutral]","[neutral, neutral]","[answer, question]",right#Your kids go to school here too,right,1,1,1,1,0
3,童文洁,"[1, 1, 0]","[neutral, neutral, neutral]","[neutral, neutral, neutral]","[statement-non-opinion, question, answer]",What a coincidence my son is here#what grade i...,What a coincidence my son is here what grade i...,0,1,1,0,1
4,童文洁,"[0, 1, 1]","[neutral, neutral, neutral]","[neutral, neutral, neutral]","[answer, statement-non-opinion, statement-non-...",class three#My son is also in class three#My s...,My son is also in class three My son is Fang Y...,0,1,1,0,1


In [5]:
df_data['uttr_len'] = df_data['dialog_state'].apply(lambda x: len(x))
df_data['uttr_len'].describe()

count    13215.000000
mean         7.793795
std          4.851597
min          1.000000
25%          4.000000
50%          6.000000
75%         10.000000
max         56.000000
Name: uttr_len, dtype: float64

In [6]:
for personality in ['A','C','E','O','N']:
    df_tmp = df_data
    df_tmp['labels'] = df_tmp[personality]
    df_tmp.to_csv('CPED_'+personality+'.tsv', sep='\t', index=False)

In [7]:
def cped_process(r):
    ans = '' 
    dialog_state = eval(r['dialog_state'])
    text_list = r['sent'].split('#')
    assert len(text_list) == len(dialog_state)
    
    for i in range(len(text_list)):
        if dialog_state[i] == 1:
            character = 'Speaker'
        else:
            character = 'Others'
        ans += character + ' : ' + text_list[i] + '; '
    return ans


def cped_get_personailty_description(r, p):
    character = 'Speaker'
    label = r['labels']
    key = p+ '_pos'
    # if label == 1:
    #     key = p+ '_pos'
    # else:
    #     key = p+ '_neg'
    return character + ' ' + personality_description[key]


def cped_get_emotion_prompt(r):
    dialog_state = eval(r['dialog_state'])
    emotions = eval(r['emotions'])
    character = 'Speaker'
    ans = ''
    for i in range(len(dialog_state)):
        if i == 0:
            if dialog_state[i] == 0:
                ans += 'First, the emotion of others is ' + emotions[i] + ', '
            elif dialog_state[i] == 1:
                ans += 'The emotion of ' + character + ' is initially ' + emotions[i] + ', '
        else:
            if dialog_state[i] == 0:
                ans += 'the emotion of others is ' + emotions[i] + ', '
            elif dialog_state[i] == 1:
                ans += character + ' respond with ' + emotions[i] + ', '
    return ans

In [8]:
personality_description = {
    'A_pos': 'is friendly, cooperative, empathetic, and compassionate, often prioritizing harmonious relationships and the well-being of others.',
    'A_neg': 'is confrontational, uncooperative, lacking empathy, and often prioritizing their own needs and desires over the well-being of others.',
    'C_pos': 'is organized, responsible, diligent, detail-oriented, and committed to achieving their goals with a strong sense of duty and self-discipline.',
    'C_neg': 'is disorganized, careless, impulsive, lacking discipline, and often displaying a disregard for responsibilities and commitments.',
    'E_pos': 'is outgoing, sociable, energetic, and thriving in social interactions, often seeking stimulation and enjoying the company of others.',
    'E_neg': 'is introverted, reserved, quiet, and often preferring solitude or smaller social settings, conserving energy and finding fulfillment in introspection and reflection.',
    'O_pos': 'has curiosity, open-mindedness, creativity, tolerance, emotional expressiveness, and willingness to embrace new experiences and ideas.',
    'O_neg': 'is closed-minded, resistant to change, lacking curiosity, intolerant of differences, emotionally guarded, and hesitant to explore new ideas or experiences.',
    'N_pos': 'is prone to experiencing negative emotions, such as anxiety, worry, and mood swings, often displaying heightened sensitivity to stress and a tendency towards self-doubt and emotional instability.',
    'N_neg': 'is emotionally stable, resilient, and composed, often displaying a calm and balanced demeanor, and having a tendency to handle stress and adversity with ease.'
}


In [9]:
personality = ['A', 'C', 'E', 'O', 'N']
for p in personality:
    print(p, '...')
    df = pd.read_csv('CPED_'+p+'.tsv', sep='\t')
    df_tmp = df[['sent', 'labels', 'emotions', 'dialog_state']]
    df_tmp['origin_sent'] = df_tmp.apply(cped_process, axis=1)
    df_tmp['personality_description'] = df_tmp.apply(cped_get_personailty_description, p=p, axis=1)
    df_tmp['affective_prompt'] = df_tmp.apply(cped_get_emotion_prompt, axis=1)
    df_tmp['sent'] = df_tmp['origin_sent']
    df_tmp.to_csv('../CPED_'+p+'_with_role_'+str(flow_len)+'.tsv', sep='\t', index=False)


A ...


/tmp/ipykernel_3189690/2651777719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['origin_sent'] = df_tmp.apply(cped_process, axis=1)
/tmp/ipykernel_3189690/2651777719.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['personality_description'] = df_tmp.apply(cped_get_personailty_description, p=p, axis=1)
/tmp/ipykernel_3189690/2651777719.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

C ...


/tmp/ipykernel_3189690/2651777719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['origin_sent'] = df_tmp.apply(cped_process, axis=1)
/tmp/ipykernel_3189690/2651777719.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['personality_description'] = df_tmp.apply(cped_get_personailty_description, p=p, axis=1)
/tmp/ipykernel_3189690/2651777719.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

E ...


/tmp/ipykernel_3189690/2651777719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['origin_sent'] = df_tmp.apply(cped_process, axis=1)
/tmp/ipykernel_3189690/2651777719.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['personality_description'] = df_tmp.apply(cped_get_personailty_description, p=p, axis=1)
/tmp/ipykernel_3189690/2651777719.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

O ...


/tmp/ipykernel_3189690/2651777719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['origin_sent'] = df_tmp.apply(cped_process, axis=1)
/tmp/ipykernel_3189690/2651777719.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['personality_description'] = df_tmp.apply(cped_get_personailty_description, p=p, axis=1)
/tmp/ipykernel_3189690/2651777719.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

N ...


/tmp/ipykernel_3189690/2651777719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['origin_sent'] = df_tmp.apply(cped_process, axis=1)
/tmp/ipykernel_3189690/2651777719.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['personality_description'] = df_tmp.apply(cped_get_personailty_description, p=p, axis=1)
/tmp/ipykernel_3189690/2651777719.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [2]:
import pandas as pd
flow_len = 1
p = 'A'
df_tmp = pd.read_csv('../CPED_'+p+'_with_role_'+str(flow_len)+'.tsv', sep='\t')
df_tmp.head()

,sent,labels,emotions,dialog_state,origin_sent,personality_description,affective_prompt
0,Speaker : what a coincidence; Speaker : the ca...,0,"['neutral', 'neutral', 'neutral', 'neutral']","[1, 1, 0, 1]",Speaker : what a coincidence; Speaker : the ca...,"Speaker is friendly, cooperative, empathetic, ...","The emotion of Speaker is initially neutral, S..."
1,Speaker : right; Others : Your kids go to scho...,1,"['neutral', 'neutral', 'neutral']","[1, 0, 1]",Speaker : right; Others : Your kids go to scho...,"Speaker is friendly, cooperative, empathetic, ...","The emotion of Speaker is initially neutral, t..."
2,Speaker : What a coincidence my son is here; S...,0,"['neutral', 'neutral', 'neutral', 'neutral']","[1, 1, 0, 1]",Speaker : What a coincidence my son is here; S...,"Speaker is friendly, cooperative, empathetic, ...","The emotion of Speaker is initially neutral, S..."
3,Others : class three; Speaker : My son is also...,0,"['neutral', 'neutral', 'neutral', 'neutral', '...","[0, 1, 1, 1, 0]",Others : class three; Speaker : My son is also...,"Speaker is friendly, cooperative, empathetic, ...","First, the emotion of others is neutral, Speak..."
4,"Speaker : Ji Yangyang, it seems that I heard F...",0,"['neutral', 'neutral', 'neutral', 'neutral', '...","[1, 1, 1, 0, 1]","Speaker : Ji Yangyang, it seems that I heard F...","Speaker is friendly, cooperative, empathetic, ...","The emotion of Speaker is initially neutral, S..."
